# Previsão de Churn e Receita para Empresa de Telecom
**Autor:** Luís Felipe Nogueira Souza
**Data:** Setembro de 2025

## 1. Introdução e Definição do Problema

Contexto de Negócio: Uma empresa de telecomunicações busca entender os fatores que levam seus clientes a cancelar o serviço (churn). O objetivo deste projeto é construir um modelo de Machine Learning para prever quais clientes têm maior probabilidade de churn.

Objetivo Final: Com base no modelo preditivo, faremos uma simulação para estimar a receita mensal da empresa nos próximos 6 meses, considerando a perda de clientes.

Fonte dos Dados: O dataset utilizado foi obtido do Kaggle e contém informações demográficas, de serviços contratados e de faturamento de cada cliente.

In [1]:
import pandas as pd

caminho_arquivo = '../data/01_raw/dataset_telecom.csv'
df = pd.read_csv(caminho_arquivo)
print("### Amostra dos Dados ###")
print(df.head())
print("\n### Informações do DataFrame ###")
df.info()

### Amostra dos Dados ###
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingM

**Análise Inicial:** O dataset possui 7043 linhas e 21 colunas. A coluna TotalCharges foi importada como texto ('object') e precisará de tratamento. Verificamos também um desbalanceamento na variável alvo Churn, onde a maioria dos clientes (aprox. 73%) não cancela o serviço.

In [2]:
# --- Limpeza e Preparação dos Dados ---
df_clean = df.copy()
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
print(f"Valores nulos em TotalCharges após conversão: {df_clean['TotalCharges'].isnull().sum()}")

df_clean['TotalCharges'] = df_clean['TotalCharges'].fillna(0)
df_clean = df_clean.drop('customerID', axis=1)
cols_to_map = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in cols_to_map:
    df_clean[col] = df_clean[col].map({'Yes': 1, 'No': 0})

print("\n### Informações após a limpeza ###")
df_clean.info()
print("\n### Amostra dos dados limpos ###")
df_clean.head()

Valores nulos em TotalCharges após conversão: 11

### Informações após a limpeza ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  Pape

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,Male,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,Male,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,Male,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


**Limpeza e Transformação:** A coluna TotalCharges foi convertida para numérica, e os valores ausentes (de clientes com 0 meses de contrato) foram preenchidos com 0. Colunas binárias foram mapeadas para 1 e 0 para facilitar a análise.

In [3]:
# --- Codificação de Variáveis Categóricas ---

df_processed = df_clean.copy()
cols_to_encode = df_processed.select_dtypes(include='object').columns
print(f"Colunas que serão codificadas: {list(cols_to_encode)}")

df_processed = pd.get_dummies(df_processed, columns=cols_to_encode, drop_first=True)
print("\n### Amostra dos dados após One-Hot Encoding ###")
pd.set_option('display.max_columns', None)
print(df_processed.head())
print("\n### Informações do DataFrame Final ###")
df_processed.info()

Colunas que serão codificadas: ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

### Amostra dos dados após One-Hot Encoding ###
   SeniorCitizen  Partner  Dependents  tenure  PhoneService  PaperlessBilling  \
0              0        1           0       1             0                 1   
1              0        0           0      34             1                 0   
2              0        0           0       2             1                 1   
3              0        0           0      45             0                 0   
4              0        0           0       2             1                 1   

   MonthlyCharges  TotalCharges  Churn  gender_Male  \
0           29.85         29.85      0        False   
1           56.95       1889.50      0         True   
2           53.85        108.15      1         True   
3           42.30       1840.75   

### Modelo Baseline: Regressão Logística

Para estabelecer uma base de comparação (baseline), iniciamos com um modelo de Regressão Logística. É um algoritmo robusto e interpretável que nos dará um primeiro indicativo da previsibilidade do problema.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# --- Construção do Modelo de Machine Learning ---

X = df_processed.drop('Churn', axis=1)
y = df_processed['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia do modelo: 0.8204

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.69      0.60      0.64       373

    accuracy                           0.82      1409
   macro avg       0.77      0.75      0.76      1409
weighted avg       0.81      0.82      0.82      1409



**Resultados do Baseline:** O modelo de Regressão Logística alcançou uma acurácia de 82% e, mais importante, um recall de 60% para a classe de churn. Nosso objetivo agora é melhorar, principalmente, o recall.

### Modelo Avançado: XGBoost

Para tentar capturar relações mais complexas nos dados e melhorar a capacidade de identificar os clientes que cancelam, vamos agora treinar um modelo de Gradient Boosting com a biblioteca XGBoost.

In [5]:
!pip install xgboost

In [6]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42, eval_metric='logloss')

xgb_model.fit(X_train_scaled, y_train)

y_pred_xgb = xgb_model.predict(X_test_scaled)

print("--- Resultados do XGBoost ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(classification_report(y_test, y_pred_xgb))

--- Resultados do XGBoost ---
Acurácia: 0.7913
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1036
           1       0.63      0.52      0.57       373

    accuracy                           0.79      1409
   macro avg       0.73      0.71      0.72      1409
weighted avg       0.78      0.79      0.78      1409



**Resultados do XGBoost:** O modelo XGBoost atingiu um recall de 52%, sem haver uma melhoria em relação ao nosso baseline. Devido a sua performance inferior, será realizado uma otimização dos hiperparâmetros para o XGBoost.

In [7]:
from sklearn.model_selection import GridSearchCV

# --- Otimização de Hiperparâmetros para o XGBoost ---
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [100, 200]
}
xgb_model_base = XGBClassifier(random_state=42, eval_metric='logloss')
grid_search = GridSearchCV(estimator=xgb_model_base, 
                           param_grid=param_grid, 
                           scoring='recall', 
                           cv=3, 
                           verbose=1)

grid_search.fit(X_train_scaled, y_train)
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")

best_xgb_model = grid_search.best_estimator_
y_pred_xgb_tuned = best_xgb_model.predict(X_test_scaled)
print("\n--- Resultados do XGBoost Otimizado ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_xgb_tuned):.4f}")
print(classification_report(y_test, y_pred_xgb_tuned))

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Melhores parâmetros encontrados: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}

--- Resultados do XGBoost Otimizado ---
Acurácia: 0.7991
              precision    recall  f1-score   support

           0       0.84      0.89      0.87      1036
           1       0.64      0.54      0.59       373

    accuracy                           0.80      1409
   macro avg       0.74      0.72      0.73      1409
weighted avg       0.79      0.80      0.79      1409



**Resultados do XGBoost (Otimizado com GridSearch):** O modelo XGBoost otimizado com GridSearch atingiu um recall de 54%, uma melhoria em relação ao nosso resultado do XGBoost sem otimização.

**Conclusão:** 
Foram testados três modelos: uma Regressão Logística como baseline, um XGBoost com parâmetros padrão e um XGBoost otimizado com GridSearchCV. Curiosamente, o modelo de Regressão Logística, apesar de mais simples, apresentou o melhor desempenho, atingindo um recall de 0.60 para a classe de churn. Isso sugere que a relação entre as features e o churn neste dataset pode ser predominantemente linear, ou que o XGBoost necessitaria de uma engenharia de features mais aprofundada ou um ajuste de hiperparâmetros mais extenso para superar o baseline. Portanto, para a simulação de receita, optou-se pelo modelo de Regressão Logística por ser o mais eficaz segundo a métrica de negócio (recall).

In [8]:
# --- Simulação de Receita Final ---

#ETAPA 1: Preparar os dados para a simulação
X_full = df_processed.drop('Churn', axis=1)
X_full_scaled = scaler.transform(X_full)
previsoes = model.predict(X_full_scaled)
df_simulation = df.copy()
df_simulation['ChurnPrediction'] = previsoes

# ETAPA 2: Simulação com taxa de perda de receita mensal
receita_dos_churners = df_simulation[df_simulation['ChurnPrediction'] == 1]['MonthlyCharges'].sum()
receita_total_completa = df_simulation['MonthlyCharges'].sum()

if receita_total_completa > 0:
    taxa_perda_mensal = receita_dos_churners / receita_total_completa
else:
    taxa_perda_mensal = 0

# Simulação mês a mês
receita_atual = receita_total_completa
print(f"Receita Mensal Atual: ${receita_atual:,.2f}")
print(f"Taxa de Perda de Receita Mensal Estimada: {taxa_perda_mensal:.2%}\n")
print("--- Previsão de Receita Corrigida para os Próximos 6 Meses ---")

for month in range(1, 7):
    perda_neste_mes = receita_atual * taxa_perda_mensal
    receita_atual -= perda_neste_mes
    print(f"Mês {month}: Receita Prevista = ${receita_atual:,.2f} (Perda de ${perda_neste_mes:,.2f})")

Receita Mensal Atual: $456,116.60
Taxa de Perda de Receita Mensal Estimada: 27.38%

--- Previsão de Receita Corrigida para os Próximos 6 Meses ---
Mês 1: Receita Prevista = $331,218.95 (Perda de $124,897.65)
Mês 2: Receita Prevista = $240,521.82 (Perda de $90,697.13)
Mês 3: Receita Prevista = $174,660.13 (Perda de $65,861.69)
Mês 4: Receita Prevista = $126,833.23 (Perda de $47,826.89)
Mês 5: Receita Prevista = $92,102.70 (Perda de $34,730.53)
Mês 6: Receita Prevista = $66,882.37 (Perda de $25,220.33)


In [12]:
import joblib
import pandas as pd
import os

os.makedirs('models/', exist_ok=True)

joblib.dump(model, '../models/modelo_churn.pkl')
joblib.dump(scaler, '../models/scaler.pkl')

import os
os.makedirs('../data/04_final/', exist_ok=True)

df.to_csv('../data/04_final/dados_originais.csv', index=False)
df_processed.to_csv('../data/04_final/dados_processados.csv', index=False)

print("Modelo, escalonador e dados salvos com sucesso!")

Modelo, escalonador e dados salvos com sucesso!
